In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as LA
import os

In [2]:
def loadData():
    pb1_file = '../data/pb/hw11pb1.csv'
    pb2_file = '../data/pb/hw11pb2.csv'
    
    pb1 = np.loadtxt(pb1_file, delimiter=',')
    pb2 = np.loadtxt(pb2_file, delimiter=',')
    
    return pb1, pb2

In [3]:
# add log probabilities
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t-1][states[0]]["prob"]*trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t-1][prev_st]["prob"]*trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
                    
            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}

    opt = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    print(' '.join(opt) )

def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

# Part 1

# a)

References: https://en.wikipedia.org/wiki/Viterbi_algorithm

In [4]:
p1 = 1.0/10.0
p2 = 1.0/2.0
p3 = 1.0/6.0

init_F = p2
init_L = p2

fair = [p3, p3, p3, p3, p3, p3]
loaded = [p1, p1, p1, p1, p1, p2]

pb1, pb2 = loadData()

obs = pb1 #('normal', 'cold', 'dizzy')
states = ('1', '2')
start_p = {'1': 0.5, '2': 0.5}
trans_p = {
   '1' : {'1': 0.95, '2': 0.05},
   '2' : {'1': 0.05, '2': 0.95}
   }
emit_p = {
   '1' : {1.: p3, 2.: p3, 3.: p3, 4.: p3, 5.: p3, 6.: p3},
   '2' : {1.: p1, 2.: p1, 3.: p1, 4.: p1, 5.: p1, 6.: p2}
   }

In [5]:
viterbi(obs,
        states,
        start_p,
        trans_p,
        emit_p)

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2


# b) 
Refrences: https://en.wikipedia.org/wiki/Forward–backward_algorithm

In [6]:
def fwd_bkw(observations, states, start_prob, trans_prob, emm_prob, end_st):
    # forward part of the algorithm
    fwd = []
    miu = []
    f_prev = {}
    for i, observation_i in enumerate(observations):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = start_prob[st]
            else:
                prev_f_sum = sum(f_prev[k]*trans_prob[k][st] for k in states)

            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum
            
            if i == 114:
                print('alpha_{k}_115 = '.format(k = st), f_curr[st])
        
        miu.append(1/(f_curr['1'] + f_curr['2']))
        
        for st in states:
            f_curr[st] *= miu[i]
        
        fwd.append(f_curr)
        f_prev = f_curr

    p_fwd = sum(f_curr[k] * trans_prob[k][end_st] for k in states)

    # backward part of the algorithm
    bkw = []
    b_prev = {}
    for i, observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = trans_prob[st][end_st]
            else:
                b_curr[st] = sum(trans_prob[st][l] * emm_prob[l][observation_i_plus] * b_prev[l] for l in states)
             
            if i == 114:
                print('beta_{k}_115 = '.format(k = st), b_curr[st])
        
        for st in states:
            b_curr[st] *= miu[len(miu)-i-1]        
                
        bkw.insert(0,b_curr)
        b_prev = b_curr

    p_bkw = sum(start_prob[l] * emm_prob[l][observations[0]] * b_curr[l] for l in states)

    # merging the two parts
    '''
    posterior = []
    for i in range(len(observations)):
        posterior.append({st: fwd[i][st] * bkw[i][st] / p_fwd for st in states})

    assert p_fwd == p_bkw
    '''
    return fwd, bkw   #, posterior

In [7]:
p1 = 1.0/10.0
p2 = 1.0/2.0
p3 = 1.0/6.0

init_F = p2
init_L = p2

fair = [p3, p3, p3, p3, p3, p3]
loaded = [p1, p1, p1, p1, p1, p2]

pb1, pb2 = loadData()

obs = tuple(pb1)
states = ('1', '2')
end_state = 'E'

start_p = {'1': 0.5, '2': 0.5}
trans_p = {
   '1' : {'1': 0.94, '2': 0.05, 'E': 0.01},
   '2' : {'1': 0.05, '2': 0.94, 'E': 0.01}
   }
emit_p = {
   '1' : {1.: p3, 2.: p3, 3.: p3, 4.: p3, 5.: p3, 6.: p3},
   '2' : {1.: p1, 2.: p1, 3.: p1, 4.: p1, 5.: p1, 6.: p2}
   }

In [19]:
t = fwd_bkw(obs,
        states,
        start_p,
        trans_p,
        emit_p, 
        end_state)

print(t[0])

alpha_1_115 =  0.01288252275350961
alpha_2_115 =  0.09127048634789424
beta_1_115 =  0.009137809132210371
beta_2_115 =  0.01815799829389702
[{'1': 0.625, '2': 0.375}, {'1': 0.7247459653317393, '2': 0.2752540346682606}, {'1': 0.7970370258665599, '2': 0.2029629741334401}, {'1': 0.8458557828169297, '2': 0.15414421718307023}, {'1': 0.8772702594011446, '2': 0.12272974059885551}, {'1': 0.8968618865660883, '2': 0.10313811343391154}, {'1': 0.6659991428376644, '2': 0.3340008571623355}, {'1': 0.7551903282023327, '2': 0.24480967179766736}, {'1': 0.8179432824123062, '2': 0.1820567175876938}, {'1': 0.8594564730194573, '2': 0.14054352698054273}, {'1': 0.6080704645870332, '2': 0.3919295354129668}, {'1': 0.33070669050595614, '2': 0.6692933094940439}, {'1': 0.4705672996835969, '2': 0.5294327003164032}, {'1': 0.5998614517572903, '2': 0.40013854824270956}, {'1': 0.7055479602816387, '2': 0.2944520397183612}, {'1': 0.7835843924927429, '2': 0.21641560750725708}, {'1': 0.8369842105539175, '2': 0.1630157894460

# Part 2

In [33]:
def baum_welch(n_iter=1000):
    V = list(obs)
    a = np.array([[0.94, 0.05, 0.01],[0.05, 0.94, 0.01]])
    b = np.array([fair, loaded])
    initial_distribution = np.array((0.5, 0.5))
    
    M = a.shape[0]
    T = len(V)
 
    for n in range(n_iter):
        #alpha = forward(V, a, b, initial_distribution)
        #beta = backward(V, a, b)
        
        alpha, beta = fwd_bkw(obs,
                                states,
                                start_p,
                                trans_p,
                                emit_p, 
                                end_state)
 
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, int(V[t + 1])].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
 
        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)
 
        b = np.divide(b, denominator.reshape((-1, 1)))
 
    return {"a":a, "b":b}

In [34]:
p1 = 1.0/10.0
p2 = 1.0/2.0
p3 = 1.0/6.0

init_F = p2
init_L = p2

fair = [p3, p3, p3, p3, p3, p3]
loaded = [p1, p1, p1, p1, p1, p2]

pb1, pb2 = loadData()

obs = tuple(pb1)
states = ('1', '2')
end_state = 'E'

start_p = {'1': 0.5, '2': 0.5}
trans_p = {
   '1' : {'1': 0.94, '2': 0.05, 'E': 0.01},
   '2' : {'1': 0.05, '2': 0.94, 'E': 0.01}
   }
emit_p = {
   '1' : {1.: p3, 2.: p3, 3.: p3, 4.: p3, 5.: p3, 6.: p3},
   '2' : {1.: p1, 2.: p1, 3.: p1, 4.: p1, 5.: p1, 6.: p2}
   }

initital = np.array((0.5, 0.5))



In [35]:
baum_welch()

alpha_1_115 =  0.01288252275350961
alpha_2_115 =  0.09127048634789424
beta_1_115 =  0.009137809132210371
beta_2_115 =  0.01815799829389702


TypeError: list indices must be integers or slices, not tuple